In [23]:
import math
import pickle
import re
import pandas as pd
from collections import Counter
from tqdm import tqdm
import random


class BigramModel:
    def __init__(self, tokens: list):
        if tokens is None:
            print("Tokens cannot be null!")
        else:
            self.tokens: list = BigramModel.add_sentence_boundaries(tokens)
            _unigram_counts = BigramModel.count_unigrams(self.tokens)
            _unigram_counts_tuples = _unigram_counts.most_common(len(_unigram_counts))
            self.unigrams = pd.DataFrame(_unigram_counts_tuples, columns=['unigram', 'count'])
            #self.unigrams.drop(self.unigrams.loc[self.unigrams['unigram'] == '</s>'].index, inplace=True)
            _bigram_counts = BigramModel.make_count_bigrams(self.tokens)
            _bigram_count_tuples = _bigram_counts.most_common(len(_bigram_counts))
            self.bigrams = pd.DataFrame(_bigram_count_tuples, columns=['bigram', 'count'])

    def __setstate__(self, state):
        self.unigrams = pd.DataFrame.from_dict(state['unigram_pickle'])
        self.bigrams = pd.DataFrame.from_dict(state['bigram_pickle'])
        self.tokens = state['tokens']

    def __getstate__(self):
        dict = {'bigram_pickle': self.bigrams.to_dict(),
                'unigram_pickle': self.unigrams.to_dict(), 'tokens': self.tokens}
        return dict

    def probability(self, w: str, w_n: str, smoothing_constant: float = 0.0):
        """
        @param w: The token we have just seen
        @param w_n: the probability of seeing token w_n
        @param smoothing_constant: the constant with which smoothing is applied
        This function calculates the probability of seeing token w_n after seeing token w
        """
        bigram: tuple = (w, w_n)
        try:
            bigram_count = \
                self.bigrams.loc[self.bigrams['bigram'] == bigram]['count'].tolist()[0]
        except:
            return 0.0
        try:
            unigram_count = \
                self.unigrams.loc[self.unigrams['unigram'] == w_n]['count'].tolist()[0]
        except:
            return 0.0
        if smoothing_constant == 0.0:
            # Locate the bigram or unigram we want the probability of
            return bigram_count / unigram_count
        else:
            total_words = len(self.unigrams)
            t = bigram_count + smoothing_constant
            n = unigram_count + smoothing_constant * total_words
            return t / n

    def perplexity(self, sent: list, smoothing_constant: float = 1.0) -> float:
        """
        @param sent: Sentence in the form of a list of tokens
        @param smoothing_constant: The constant used to apply smoothing
        This calculates the perplexity of the given sentence.
        """
        sent_copy = sent.copy()
        sent.insert(0, '<s>')
        sent.append('</s>')
        #sent_copy.remove('</s>')
        n = len(sent_copy)
        probs = []
        for i in range(n - 1):
            probability = self.probability(sent_copy[i], sent_copy[i + 1], smoothing_constant)
            q = 1 / probability
            probs.append(q)
        s = math.prod(probs)
        return s ** (1 / n)

    def choose_successor(self, word: str, smoothing_constant: float = 0.0) -> str | None:
        """
        @param word: The word/token for which to generate the successor
        @param smoothing_constant: The constant used to apply smoothing
        This function uses the probability function to randomly but based on probabilities
        choose a successor to the token given.
        """
        try:
            unigram_count = \
                self.unigrams.loc[self.unigrams['unigram'] == word]['count'].tolist()[0]
        except:
            return None

        possible_bigrams = \
            self.bigrams[self.bigrams['bigram'].apply(lambda x: x[0] == word)][
                'bigram'].tolist()
        prob2 = []
        for bigram in tqdm(possible_bigrams, ncols=100, desc=f"Choosing successor for: {word}"):
            prob2.append(self.probability(word, bigram[1], 0.0))
        successor: tuple = random.choices(possible_bigrams, weights=prob2, k=1)
        return successor[0][1]

    def save_model(self, location: str):
        """
        @param location: Path to where the model should be stored
        Uses pickle to store the model into a file to be loaded back in later
        """
        with open(location, 'wb') as f:
            pickle.dump(self, f)

    @staticmethod
    def load_model(location: str):
        """
        @param location: Path to where the model is stored
        Uses to pickle to load a store model back in to e.g. a variable.
        """
        with open(location, 'rb') as f:
            return pickle.load(f)

    @staticmethod
    def count_unigrams(tokens: list) -> Counter:
        """
        @param tokens: list of tokenized sentences
        Takes a list of tokenized sentences and generates the appropriate unigrams and counts them
        """
        totWords: list = []
        for p, words in enumerate(
                tqdm(tokens, ncols=100, desc='Making and counting Unigrams')):  # tqdm prints a progressbar
            for word in words:
                totWords.append(word)
        return Counter(totWords)

    @staticmethod
    def make_count_bigrams(tokens: list) -> Counter:
        """
        @param tokens: list of tokenized sentences
        Takes a list of tokenized sentences and generates the appropriate bigrams and counts them
        """
        # bigram_counts = Counter()
        bigrams: list = []
        for p, words in enumerate(tqdm(tokens, ncols=100, desc='Making and counting Bigrams')):  # tqdm prints a progressbar
            #words.remove("</s>")
            for i in range(len(words) - 1):
                bigrams.append((words[i], words[i + 1]))
        return Counter(bigrams)

    @staticmethod
    def add_sentence_boundaries(tokens: list) -> list:
        """
        @param tokens: list of tokenized sentences
        Takes a list of tokenized sentences and adds sentence boundaries to all the sentences
        """
        tokens_without_punctuation = BigramModel.remove_punctuation_tokens(tokens)
        tokens_with_boundaries: list = []
        item: list
        for i, item in enumerate(
                tqdm(tokens_without_punctuation, ncols=100, desc='Adding boundaries')):  # tqdm prints a progressbar
            item.insert(0, "<s>")
            item.append("</s>")
            tokens_with_boundaries.append(item)
        return tokens_with_boundaries

    @staticmethod
    def remove_punctuation_tokens(tokens: list):
        """
        @param tokens: list of tokenized sentences
        Takes a list of tokenized sentences and removes the tokens that solemnly consist of punctuation from the list
        """
        to_return: list = []
        for sent in tokens:
            to_return.append([p.lower() for p in sent if not re.match('\W', p)])
        return to_return


In [24]:
#import Model
from corpusreader import CorpusReader
import pandas as pd

reader = CorpusReader("./train")
model = BigramModel(reader.sents())

Making and counting Bigrams: 100%|████████████████████████| 11909/11909 [00:00<00:00, 134705.74it/s]


In [6]:
successor = model.choose_successor('<s>')

Choosing successor for: <s>: 100%|████████████████████████████████| 824/824 [00:14<00:00, 58.77it/s]


In [7]:
print(successor)

consequently


In [35]:
model.bigrams[model.bigrams['bigram'].apply(lambda x: '</s>' in x[1])]['bigram']

32            (it, </s>)
58           (him, </s>)
76           (you, </s>)
132         (them, </s>)
160           (me, </s>)
              ...       
96332    (governs, </s>)
96339      (cruel, </s>)
96401      (wheel, </s>)
96405    (thrones, </s>)
96429    (accused, </s>)
Name: bigram, Length: 3989, dtype: object

In [25]:
def generate_sentence(model: BigramModel, smoothing_constant=0.0):
    sent = ['<s>']
    while sent[len(sent) - 1] != '</s>':
        successor = model.choose_successor(sent[len(sent) - 1], smoothin)
        sent.append(successor)
    return sent

In [26]:
sent = generate_sentence(model, smoothing_constant=1.0)

Choosing successor for: muddle: 100%|█████████████████████████████████| 1/1 [00:00<00:00, 50.05it/s]


In [31]:
print(model.choose_successor('insincerity', smoothing_constant=5.0))

Choosing successor for: insincerity: 100%|████████████████████████████| 1/1 [00:00<00:00, 58.42it/s]

</s>


In [32]:
def generate_sentence(model: BigramModel, smoothing_constant=0.0):
    sent = ['<s>']
    while sent[len(sent) - 1] != '</s>':
        successor = model.choose_successor(sent[len(sent) - 1], smoothing_constant=smoothing_constant)
        sent.append(successor)
    return sent

In [33]:
sent = generate_sentence(model, 1.0)

Choosing successor for: wars: 100%|███████████████████████████████████| 2/2 [00:00<00:00, 60.47it/s]
